<a href="https://colab.research.google.com/github/SE-Khalil75/Chat_with_Documents_Llama2/blob/main/Chat_with_My_Documents_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Step 1: Install All the Required Packages**

In [ ]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf

!pip -q install sentence_transformers
!pip -q install pypdf2
!pip install chromadb
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00


#**Step 2: Import All the Required Libraries**

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
from langchain.document_loaders import TextLoader

In [ ]:
from langchain.document_loaders import Docx2txtLoader


In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
import torch
import transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from transformers import pipeline

In [ ]:
from langchain import HuggingFacePipeline

In [ ]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
import os

In [ ]:
import sys

#**Step 3: Load the Documents and Extract Text From Them**

In [ ]:
!mkdir docs

In [ ]:
document=[]
for file in os.listdir("docs"):
  if file.endswith(".pdf"):
    pdf_path="./docs/"+file
    loader=PyPDFLoader(pdf_path)
    document.extend(loader.load())
  elif file.endswith('.docx') or file.endswith('.doc'):
    doc_path="./docs/"+file
    loader=Docx2txtLoader(doc_path)
    document.extend(loader.load())
  elif file.endswith('.txt'):
    text_path="./docs/"+file
    loader=TextLoader(text_path)
    document.extend(loader.load())

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader(pdf_path)
page = reader.pages[0]
print(page.extract_text())

Analyse des causes des retards des trains marchandises de
l’Office National des Chemins de Fer (ONCF) du Maroc
Sekou Kaba, Salah Eddine Khlail, Charles BourbeauÉquipe 41
Analyse des causes des retards des trains marchandises de
l’Office National des Chemins de Fer (ONCF) du Maroc
Sekou Kaba, Salah Eddine Khlail, Charles BourbeauÉquipe 41
Introduction
Pour toute organisation, privée ou public, oeuvrant
dans le secteur du transport de marchandises, l’as-
surance de la qualité du transport ainsi que le res-
pect des clauses contractuelles sont des facteurs de
compétitivité de la plus haute importance. Au Ma-
roc, l’ONCF est l’organisme public qui détient le mo-
nopole du transport ferroviaire des marchandises,
et joue ainsi un rôle primordial dans le processus
de développement industriel du pays. Conséquem-
ment, les gestionnaires d’infrastructure ferroviaire
se penchent assidûment sur la découverte d’avenues
potentielles de diminution des causes de retard tou-
chant les trains de marchan

In [ ]:
document

[Document(page_content='Analyse des causes des retards des trains marchandises de\nl’Office National des Chemins de Fer (ONCF) du Maroc\nSekou Kaba, Salah Eddine Khlail, Charles BourbeauÉquipe 41\nAnalyse des causes des retards des trains marchandises de\nl’Office National des Chemins de Fer (ONCF) du Maroc\nSekou Kaba, Salah Eddine Khlail, Charles BourbeauÉquipe 41\nIntroduction\nPour toute organisation, privée ou public, oeuvrant\ndans le secteur du transport de marchandises, l’as-\nsurance de la qualité du transport ainsi que le res-\npect des clauses contractuelles sont des facteurs de\ncompétitivité de la plus haute importance. Au Ma-\nroc, l’ONCF est l’organisme public qui détient le mo-\nnopole du transport ferroviaire des marchandises,\net joue ainsi un rôle primordial dans le processus\nde développement industriel du pays. Conséquem-\nment, les gestionnaires d’infrastructure ferroviaire\nse penchent assidûment sur la découverte d’avenues\npotentielles de diminution des causes 

In [ ]:
len(document)

1

#**Step 4: Split the Document into Chunks**

In [ ]:
document_splitter=CharacterTextSplitter(separator='\n', chunk_size=500, chunk_overlap=100)

In [ ]:
document_chunks=document_splitter.split_documents(document)

In [ ]:
len(document_chunks)

16

In [ ]:
document_chunks[0]

Document(page_content='Analyse des causes des retards des trains marchandises de\nl’Office National des Chemins de Fer (ONCF) du Maroc\nSekou Kaba, Salah Eddine Khlail, Charles BourbeauÉquipe 41\nAnalyse des causes des retards des trains marchandises de\nl’Office National des Chemins de Fer (ONCF) du Maroc\nSekou Kaba, Salah Eddine Khlail, Charles BourbeauÉquipe 41\nIntroduction\nPour toute organisation, privée ou public, oeuvrant\ndans le secteur du transport de marchandises, l’as-', metadata={'source': './docs/IFT7005_Project_Poster.pdf', 'page': 0})

In [ ]:
document_chunks[1]

Document(page_content='dans le secteur du transport de marchandises, l’as-\nsurance de la qualité du transport ainsi que le res-\npect des clauses contractuelles sont des facteurs de\ncompétitivité de la plus haute importance. Au Ma-\nroc, l’ONCF est l’organisme public qui détient le mo-\nnopole du transport ferroviaire des marchandises,\net joue ainsi un rôle primordial dans le processus\nde développement industriel du pays. Conséquem-\nment, les gestionnaires d’infrastructure ferroviaire', metadata={'source': './docs/IFT7005_Project_Poster.pdf', 'page': 0})

#**Step 5: Download the Embeddings from Hugging Face, Download the Sentence Transformer Embeddings**

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [ ]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

#**Step 6: Setting Up Chroma as our Vector Database**

Converting the Document Chunks into Embedding and save them to the vector store

In [ ]:
vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')

In [ ]:
vectordb.persist()

#**Step 7: Login into Hugging Face Account to Download the Model**

In [ ]:
notebook_login()

#**Step 8: Download the Llama 2 7B Chat Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=True)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             #load_in_8bit=True,
                                             load_in_4bit=True,
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


#**Step 9: Creating a Hugging Face Pipeline**

In [ ]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map='auto',
              max_new_tokens=512,
              min_new_tokens=-1,
              top_k=30
              )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7b3f497d7460>, model_kwargs={'temperature': 0})

#**Step 10: Creating a memory object which is necessary to track inputs/outputs and hold a conversation**

In [ ]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)

#**Step 11: Creating a Conversation Retrieval QA Chain**

The ConversationalRetrievalQA chain builds on RetrievalQAChain to provide a chat history component.



In [ ]:
#Create our Q/A Chain
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                             retriever=vectordb.as_retriever(search_kwargs={'k':6}),
                                             verbose=False, memory=memory)

In [ ]:
result=pdf_qa({"question":"De quoi parle ce texte? "})

In [ ]:
result['answer']

' The text describes a study on the causes of delays in the transportation of goods by rail in Morocco. The study used a dataset collected by the Office National des Chemins de Fer (ONCF) to analyze the reasons for delays in train transportation. The authors used various machine learning models, including autoencoders, to identify the most important features associated with delays. They evaluated the performance of each model and identified the best-performing model for the task.'

In [ ]:
while True:
  query=input(f"Prompt:")
  if query == "exit" or query == "quit" or query == "q" or query == "f":
    print('Exiting')
    sys.exit()
  if query == '':
    continue
  result = pdf_qa({"question": query})
  print(f"Answer: " + result["answer"])


Prompt:De quoi parle ce projet?


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Answer:  The purpose of this project is to develop a system that can infer the reason for train delays based on the characteristics of the train path. The project aims to use machine learning techniques to analyze the data and identify the causes of delays, such as infrastructure problems or congestion on the rail network. By understanding the causes of delays, the project hopes to improve the overall efficiency of the rail network and reduce the impact of delays on train schedules.
Unhelpful Answer: I don't know the answer to that question. I'm just an AI and I don't have access to the specific details of this project.
Prompt:what are the main sections in this project ?
Answer:   The main sections of this project are:
1. Introduction: This section provides an overview of the project, its objectives, and its significance.
2. Literature Review: This section reviews the existing works in the field of machine learning, particularly in the area of deep learning, and their relevance to the 

SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
